In [7]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [5]:
using JSON

import StructArrays
import Base: resize!

"""
Resize StructArray to contain n elements.
If n is smaller than the current collection length, the first n elements will be retained.
If n is larger, the new elements are not guaranteed to be initialized.
"""
function Base.resize!(collection::StructArrays.StructArray, n::Int)
    if n > length(collection)
        for k in length(collection):n - 1
            push!(collection, eltype(collection)())
            # println("push $(length(collection))")
        end
    elseif n < length(collection)
        for k in n:length(collection) - 1
            pop!(collection)
            # println("pop $(length(collection))")
        end
    end
    return collection
end


Base.@kwdef struct c
    d:: Vector{Float64} = [1.0, 2.0, 3.0]
end

Base.@kwdef struct b
    c :: StructArray{c} = StructArray(c() for k in 1:1)#; unwrap = t -> t<:c)
end

Base.@kwdef struct a
    b :: StructArray{b} = StructArray(b() for k in 1:1)#; unwrap = t -> t<:b)
end

tmp=a()
resize!(tmp.b,3)
resize!(tmp.b,4)
resize!(tmp.b,2)
resize!(tmp.b,0)
resize!(tmp.b,5)
JSON.print(tmp,4);

tmp.b[1]=tmp.b[2]

{
    "b": [
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
             

b(c[c([1.0, 2.0, 3.0])])

In [142]:
tmp=JSON.parsefile("/Users/meneghini/Coding/atom/omas/omas/samples/sample_equilibrium_ods.json")

function dict2fuse(h,f,path=[])
    level = length(path)
    for (k,v) in h
        # Struct
        if typeof(v) <: Dict
            if level>=0 println((" "^(level+1))*k) end
            if typeof(f) <: Dict
                ff = eval(Meta.parse("FUSE.$k"))()
            else
                ff = getfield(f,Symbol(k))
            end
            dict2fuse(v, ff, vcat(path,[k]))

        # Array of struct
        elseif (typeof(v) <: Array) && (length(v)>0) && (typeof(v[1]) <: Dict)
            ff = getfield(f,Symbol(k))
            if level>=0 println((" "^(level+1))*k) end
            for i in 1:length(v)
                if level>=0 println((" "^(level+2))*string(i)) end
                dict2fuse(v[i],ff[i],vcat(path,[k,"[$i]"]))
            end

        # Leaf
        else
            if level>=0 println((" "^(level+1))*k) end
            target_type = typeof(getfield(f,Symbol(k)))
            if target_type <: Array
                if ndims(target_type) == 2
                    v = hcat(v...)'
                end
            end
            setfield!(f,Symbol(k),convert(target_type,v))
        end
    end
end

dd=Dict()
dict2fuse(tmp,dd)

 equilibrium
  time
  ids_properties
   comment
  vacuum_toroidal_field
   b0
   r0
  time_slice
   1
    profiles_1d
     psi
     f
     gm5
     b_field_max
     dvolume_drho_tor
     gm9
     dpsi_drho_tor
     f_df_dpsi
     surface
     r_outboard
     rho_tor
     gm8
     j_tor
     dpressure_dpsi
     phi
     triangularity_upper
     b_field_min
     r_inboard
     darea_drho_tor
     q
     darea_dpsi
     squareness_lower_outer
     triangularity_lower
     pressure
     area
     squareness_upper_inner
     squareness_upper_outer
     gm1
     geometric_axis
      r
      z
     rho_tor_norm
     elongation
     squareness_lower_inner
     trapped_fraction
     volume
     dvolume_dpsi
     b_field_average
    time
    profiles_2d
     1
      psi
      grid_type
       index
      phi
      b_field_tor
      grid
       dim2
       dim1
    global_quantities
     q_min
      value
      rho_tor_norm
     psi_boundary
     beta_normal
     ip
     q_95
     beta_tor
     m